- 在上一节中，我们简单地实现了一个十分十分基础甚至有些简陋的GPT，同时起生成效果看起来也有很大的提升空间
- 这一节中，我们将对通过一系列的推导来向大家引入可以增强性能的自注意力机制


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## 自注意力机制怎么增强性能

在此之前，nn.Embedding致力于将现有的编码转化为其对应的下一位的编码

但是一个很重要的点是其忽略了现有的编码中彼此之间的联系，

如果可以利用好这份联系，使得每个字的编码可以**相互通信**，是从能产生更好的性能呢

In [2]:
# 此时我们以一个真实的情况为例，通过随机生成一些数据来代表当前的真实情况
torch.manual_seed(42)   # 设置固定的种子，使得结果可以及时的复现
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

- 最简单的通信方式，第五个编码可以很简单地与收到前面四个编码的平均影响，虽然这种通信的方式听起来也十分地弱

In [3]:
# 第一种方式：为了循环和聚合，我们使用torch.mean函数进行操作

xbow = torch.zeros((B,T,C))

for b in range(B):  # 遍历所有的batch
    for t in range(T):
        # 使用切片操作x[b,:t+1]来获取x在第b个批次中前t+1个时间步的所有元素，得到一个形状为(t,C)的张量xprev。
        xprev = x[b,:t+1] # (t,C)
        # 使用torch.mean(xprev, 0)来计算xprev在第一个维度（dim=0）上的平均值。这个操作会返回一个形状为(C,)的张量，它的每个元素是xprev在对应列上的元素的平均值。
        xbow[b,t] = torch.mean(xprev, 0) 
        
        # print(b) if b==0 else None
        # print(t) if b==0 else None
        print(xprev) if b==0 else None   # 前面的累加
        print('这是上面的均值累加：')
        print(xbow[b,t]) if b==0 else None
        print("----------------------")

# 虽然这样是可以行的，但是这里的实现方式明显可以进一步改进

tensor([[1.9269, 1.4873]])
这是上面的均值累加：
tensor([1.9269, 1.4873])
----------------------
tensor([[ 1.9269,  1.4873],
        [ 0.9007, -2.1055]])
这是上面的均值累加：
tensor([ 1.4138, -0.3091])
----------------------
tensor([[ 1.9269,  1.4873],
        [ 0.9007, -2.1055],
        [ 0.6784, -1.2345]])
这是上面的均值累加：
tensor([ 1.1687, -0.6176])
----------------------
tensor([[ 1.9269,  1.4873],
        [ 0.9007, -2.1055],
        [ 0.6784, -1.2345],
        [-0.0431, -1.6047]])
这是上面的均值累加：
tensor([ 0.8657, -0.8644])
----------------------
tensor([[ 1.9269,  1.4873],
        [ 0.9007, -2.1055],
        [ 0.6784, -1.2345],
        [-0.0431, -1.6047],
        [-0.7521,  1.6487]])
这是上面的均值累加：
tensor([ 0.5422, -0.3617])
----------------------
tensor([[ 1.9269,  1.4873],
        [ 0.9007, -2.1055],
        [ 0.6784, -1.2345],
        [-0.0431, -1.6047],
        [-0.7521,  1.6487],
        [-0.3925, -1.4036]])
这是上面的均值累加：
tensor([ 0.3864, -0.5354])
----------------------
tensor([[ 1.9269,  1.4873],
        [ 0.9007

In [4]:
# 方法2：使用矩阵乘法

a = torch.tril(torch.ones(3, 3))   # 创建一个下三角的矩阵的函数
# torch.sum 计算张量a在每一横行上的值，
a = a / torch.sum(a, 1, keepdim=True)    # ！ 这一步相当于是在
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

#  在这个过程中，实现了字符根据权重得到最终的结果

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[0., 1.],
        [3., 0.],
        [1., 1.]])
--
c=
tensor([[0.0000, 1.0000],
        [1.5000, 0.5000],
        [1.3333, 0.6667]])


* 这个时候可以揭晓，我们所想要平均的一直是**权重**

In [5]:
weight = torch.tril(torch.ones(T, T))
weight = weight / weight.sum(1, keepdim=True)  # 直接是在第一个维度上进行加和
weight
# 而在这个例子中的b，其实是x
xbow2 = weight @ x  # (B,T,T) @ (B,T,C)   ------>   (B,T,C)
torch.allclose(xbow,xbow2)   # 这个是用于检测两个张量是否在一定的容忍度内是相等的

True

结果为`True`，所以说明这样几行就解决了上面这个循环要做的事情

* 然而，这里还有一种更为巧妙的方式可以实现

In [6]:

# 第三种方式：使用softmax
trils = torch.tril(torch.ones(T,T))

weight = torch.zeros((T,T))  # 构造一个全为0的向量
weight = weight.masked_fill(trils == 0,float('-inf'))  # 使所有tril为0的位置都变为无穷大
# 然后，我们选择在每行的维度上去使用sotfmax，
weight = F.softmax(weight,dim=-1)

xbow3 = weight @ x

torch.allclose(xbow,xbow3)   # 这个是用于检测两个张量是否在一定的容忍度内是相等的


True

In [9]:
# 第4种方式 : 自注意力方式
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# 一个简单的单头注意力机制示范
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [8]:
out[0]

tensor([[[-1.5713e-01,  8.8009e-01,  1.6152e-01, -7.8239e-01, -1.4289e-01,
           7.4676e-01,  1.0068e-01, -5.2395e-01, -8.8726e-01,  1.9068e-01,
           1.7616e-01, -5.9426e-01, -4.8124e-01, -4.8598e-01,  2.8623e-01,
           5.7099e-01],
         [ 6.7643e-01, -5.4770e-01, -2.4780e-01,  3.1430e-01, -1.2799e-01,
          -2.9521e-01, -4.2962e-01, -1.0891e-01, -4.9282e-02,  7.2679e-01,
           7.1296e-01, -1.1639e-01,  3.2665e-01,  3.4315e-01, -7.0975e-02,
           1.2716e+00],
         [ 4.8227e-01, -1.0688e-01, -4.0555e-01,  1.7696e-01,  1.5811e-01,
          -1.6967e-01,  1.6217e-02,  2.1509e-02, -2.4903e-01, -3.7725e-01,
           2.7867e-01,  1.6295e-01, -2.8951e-01, -6.7610e-02, -1.4162e-01,
           1.2194e+00],
         [ 1.9708e-01,  2.8561e-01, -1.3028e-01, -2.6552e-01,  6.6781e-02,
           1.9535e-01,  2.8073e-02, -2.4511e-01, -4.6466e-01,  6.9287e-02,
           1.5284e-01, -2.0324e-01, -2.4789e-01, -1.6213e-01,  1.9474e-01,
           7.6778e-01],
    